In [1]:
import os
import sys

from google.colab import drive
drive.mount('/content/drive')

!pip install -e "/content/drive/MyDrive/Ricerca/latent-diffusion"



!pip install -q kornia
!pip install torchmetrics==0.6.0
!pip install -e ./taming-transformers
!pip install -q pandas pyarrow
!pip install git+https://github.com/openai/CLIP.git

if not os.path.exists('taming-transformers'):
    !git clone https://github.com/CompVis/taming-transformers.git

if not os.path.exists('CLIP'):
    !git clone https://github.com/openai/CLIP.git


%cd /content/drive/MyDrive/Ricerca/latent-diffusion

! pip install -r requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Obtaining file:///content/drive/MyDrive/Ricerca/latent-diffusion
  Preparing metadata (setup.py) ... done
  Attempting uninstall: latent-diffusion
    Found existing installation: latent-diffusion 0.0.1
    Uninstalling latent-diffusion-0.0.1:
      Successfully uninstalled latent-diffusion-0.0.1
  Running setup.py develop for latent-diffusion
  Using cached torchmetrics-0.6.0-py3-none-any.whl.metadata (20 kB)
Using cached torchmetrics-0.6.0-py3-none-any.whl (329 kB)
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.8.2
    Uninstalling torchmetrics-1.8.2:
      Successfully uninstalled torchmetrics-1.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.6.1 requires torchmetrics>0.

In [2]:
#!wget -O models/ldm/inpainting_big/last.ckpt https://heibox.uni-heidelberg.de/f/4d9ac7ea40c64582b7c9/?dl=1

In [ ]:
!mkdir -p /content/checkpoints

!python main.py --base models/ldm/inpainting_big/config.yaml \
                --resume models/ldm/inpainting_big/last.ckpt \
                --save_freq=20 \
                --epochs=700 \
                --stage 1 -t --gpus=0, \
                --logdir /content/drive/MyDrive/Ricerca/latent-diffusion/logs_backup \
                --lora_path=logs_backup/2026-02-06T02-57-19_config/checkpoints/last.ckpt

#!cp -r /content/checkpoints /content/drive/MyDrive/Ricerca/latent-diffusion/logs_backup

Config:
model:
  base_learning_rate: 1.0e-06
  target: ldm.models.diffusion.ddpm.LatentDiffusion
  params:
    linear_start: 0.0015
    linear_end: 0.0205
    log_every_t: 100
    timesteps: 1000
    loss_type: l1
    first_stage_key: image
    cond_stage_key: masked_image
    image_size: 64
    channels: 3
    concat_mode: true
    monitor: val/loss
    scheduler_config:
      target: ldm.lr_scheduler.LambdaWarmUpCosineScheduler
      params:
        verbosity_interval: 0
        warm_up_steps: 1000
        max_decay_steps: 50000
        lr_start: 0.001
        lr_max: 0.1
        lr_min: 0.0001
    unet_config:
      target: ldm.modules.diffusionmodules.openaimodel.UNetModel
      params:
        image_size: 64
        in_channels: 7
        out_channels: 3
        model_channels: 256
        attention_resolutions:
        - 8
        - 4
        - 2
        num_res_blocks: 2
        channel_mult:
        - 1
        - 2
        - 3
        - 4
        num_heads: 8
        resblock_u

In [ ]:
#!cp -r /content/checkpoints/2026-02-06T01-25-59_config/checkpoints/last.ckpt /content/drive/MyDrive/Ricerca/latent-diffusion/logs_backup